In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
data = pd.read_csv('/content/drive/My Drive/depressionrnn/sent_numeric_final.csv')

In [3]:
data

,condition,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,0,0.001627,0.057593,0.014496,0.557964,0.006982,0.012979,0.047923,0.001594,0.000548,0.002371,0.010774,0.016187,0.003044,0.000865,0.006406,0.002740,0.015404,0.002702,0.005829,0.007469,0.005230,0.002537,0.021831,0.002972,0.000905,0.025278,0.119680,0.001273,0.197433,0.027596,0.042909,0.028412,0.003445,0.144661,0.010809,0.152555
1,1,0.018625,0.059010,0.015516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0,0.001601,0.673036,0.000351,0.011216,0.537889,0.422260,0.713440,0.001468,0.590326,0.031161,0.064264,0.374536,0.179550,0.904945,0.002335,0.006470,0.022470,0.000595,0.004649,0.147445,0.002656,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1,0.901723,0.758650,0.918634,0.400417,0.085800,0.966300,0.991832,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0,0.025497,0.003074,0.002334,0.001891,0.010400,0.000666,0.005096,0.035518,0.022665,0.078190,0.024544,0.005663,0.087324,0.001214,0.014483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1742,0,0.010756,0.022672,0.064848,0.009764,0.003504,0.001975,0.045825,0.009806,0.000739,0.011539,0.041411,0.008914,0.139236,0.692170,0.007354,0.001140,0.013954,0.009711,0.117604,0.006017,0.015846,0.000695,0.002400,0.014634,0.918869,0.030984,0.008268,0.006418,0.008598,0.000600,0.018710,0.000774,0.004047,0.000155,0.003306,0.007298
1743,0,0.002703,0.004468,0.019325,0.013272,0.021611,0.004587,0.001012,0.000201,0.002452,0.010408,0.003858,0.000810,0.002019,0.003105,0.434770,0.002884,0.012024,0.008344,0.003646,0.003827,0.002317,0.036637,0.000673,0.089411,0.003610,0.048780,0.004619,0.001183,0.002801,0.005719,0.494916,0.660741,0.006066,0.007334,0.006883,0.008321
1744,1,0.384833,0.759091,0.988976,0.836494,0.955679,0.017949,0.924520,0.200688,0.002150,0.805992,0.058129,0.030758,0.008038,0.004497,0.004414,0.179881,0.004874,0.169836,0.935106,0.010093,0.643853,0.012858,0.000352,0.007111,0.935400,0.909491,0.848957,0.805868,0.037851,0.022139,0.009748,0.944626,0.008024,0.981319,0.113276,0.010401
1745,1,0.021663,0.024507,0.009037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [4]:
data.dtypes

condition      int64
0            float64
1            float64
2            float64
3            float64
4            float64
5            float64
6            float64
7            float64
8            float64
9            float64
10           float64
11           float64
12           float64
13           float64
14           float64
15           float64
16           float64
17           float64
18           float64
19           float64
20           float64
21           float64
22           float64
23           float64
24           float64
25           float64
26           float64
27           float64
28           float64
29           float64
30           float64
31           float64
32           float64
33           float64
34           float64
35           float64
dtype: object

In [0]:
Y=data.loc[:,'condition']

In [6]:
Y.shape

(1747,)

In [0]:
X=data.loc[:,'0':]

In [8]:
X.shape

(1747, 36)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [0]:
import torch
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
Xtrain_ = torch.tensor(x_train.values).float()
Xtest_ = torch.tensor(x_test.values).float()
Ytrain_ = torch.tensor(y_train.values).view(1,-1)[0]
Ytest_ = torch.tensor(y_test.values).view(1,-1)[0]

In [0]:
input_size = 36
output_size = 2
hidden_size = 20

In [0]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, hidden_size) 
        #self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size) 
    
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        #x = F.sigmoid(self.fc3(x))
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=-1)

In [0]:
model = Net()

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

loss_fn = nn.NLLLoss()

In [0]:
epoch_data = []
epochs = 401

for epoch in range(1, epochs):

    optimizer.zero_grad()
    Ypred = model(Xtrain_)

    loss = loss_fn(Ypred , Ytrain_)
    loss.backward()

    optimizer.step()
        
    Ypred_test = model(Xtest_)
    loss_test = loss_fn(Ypred_test, Ytest_)
    
    _,pred = Ypred_test.data.max(1)
    
    accuracy = pred.eq(Ytest_.data).sum().item() / y_test.values.size
    epoch_data.append([epoch, loss.data.item(), loss_test.data.item(), accuracy])
    
    if epoch % 100 == 0:
        print ('epoch - %d (%d%%) train loss - %.2f test loss - %.2f accuracy - %.4f'\
               % (epoch, epoch/150 * 10 , loss.data.item(), loss_test.data.item(), accuracy))

epoch - 100 (6%) train loss - 0.69 test loss - 0.70 accuracy - 0.4714
epoch - 200 (13%) train loss - 0.69 test loss - 0.69 accuracy - 0.4771
epoch - 300 (20%) train loss - 0.69 test loss - 0.69 accuracy - 0.5057
epoch - 400 (26%) train loss - 0.69 test loss - 0.69 accuracy - 0.4829


In [0]:
sent=[]
for i in range(18):
  sent.append(1)
  sent.append(0)

In [0]:
sent

[1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0]

In [0]:
sample = np.array(sent)
sample_tensor = torch.from_numpy(sample).float()

In [0]:
out = model(sample_tensor)
_,predicted = torch.max(out.data,-1)

In [0]:
predicted.item()

1

In [0]:
torch.save(model, '/content/drive/My Drive/depressionrnn/para50.pt')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
